In [2]:
import tkinter as tk
from tkinter import filedialog, scrolledtext, ttk, messagebox
import os
import threading
import ollama
from datetime import datetime
import queue

# Istoricul conversației cu modelul
conversation_history = []
stop_event = threading.Event()
message_queue = queue.Queue()

# Funcție pentru salvarea istoricului conversației
def save_conversation():
    if not os.path.exists('history'):
        os.makedirs('history')
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"history/{timestamp}.txt"
    with open(filename, "w") as f:
        for message in conversation_history:
            role = message['role']
            content = message['content']
            f.write(f"{role}: {content}\n\n")

# Funcție pentru încărcarea și analizarea fișierului
def load_file():
    filepath = filedialog.askopenfilename(title="Selectează un fișier")
    if not filepath:
        return

    try:
        with open(filepath, "rb") as f:
            file_bytes = f.read()

        file_size = os.path.getsize(filepath)
        file_name = os.path.basename(filepath)
        macro_info = f"Nume fișier: {file_name}\nDimensiune: {file_size} bytes\n"
        hex_dump = file_bytes[:256].hex()
        micro_info = f"Hex dump (primele 256 de octeți):\n{hex_dump}\n"
        analysis_info = macro_info + "\n" + micro_info

        message_queue.put(("insert", analysis_text, "Fișier încărcat cu succes:\n"))
        message_queue.put(("insert", analysis_text, analysis_info))

        # Pregătim prompt-ul și actualizăm istoricul conversației
        prompt = (
            "Ești un expert în analiza malware. Analizează următoarele detalii "
            "ale unui fișier și determină dacă este suspect sau chiar malware. "
            "Oferă un rezumat la nivel macro și micro și justifică evaluarea ta.\n\n"
            f"{analysis_info}"
        )
        conversation_history.append({'role': 'user', 'content': prompt})

        # Apel streaming în thread separat
        stop_event.clear()
        threading.Thread(
            target=call_model_stream,
            args=(analysis_text, "\nAnaliza modelului (streaming):\n"),
            daemon=True
        ).start()
    except Exception as e:
        message_queue.put(("insert", analysis_text, f"\n\nEroare: {e}"))

# Funcție care construiește și transmite cereri non-streaming (chat simplu)
def send_chat():
    user_msg = chat_entry.get().strip()
    if not user_msg:
        return
    # Afișează mesajul utilizator
    message_queue.put(("insert", chat_history, f"Tu: {user_msg}\n"))
    chat_entry.delete(0, tk.END)

    # Adăugăm în istoricul conversației și trimitem
    conversation_history.append({'role': 'user', 'content': user_msg})
    stop_event.clear()
    threading.Thread(
        target=call_model,
        args=(chat_history,),
        daemon=True
    ).start()

# Funcție pentru a șterge istoricul conversației
def clear_history():
    global conversation_history
    conversation_history = []
    message_queue.put(("delete", chat_history))
    message_queue.put(("delete", analysis_text))
    messagebox.showinfo("Istoric șters", "Istoricul conversației a fost șters.")

# Funcție pentru a opri conversația curentă
def stop_conversation():
    stop_event.set()
    messagebox.showinfo("Conversație oprită", "Conversația a fost oprită.")

# Non-streaming: răspuns complet de la model
def call_model(text_widget):
    try:
        response = ollama.chat(
            model='mistral',
            messages=conversation_history,
            stream=False
        )
        content = response.get('message', {}).get('content', '')
        # Adăugăm răspunsul în istoric
        conversation_history.append({'role': 'assistant', 'content': content})
        message_queue.put(("insert", text_widget, f"Model: {content}\n"))
        save_conversation()
    except Exception as e:
        message_queue.put(("insert", text_widget, f"Eroare la chat: {e}\n"))

# Streaming: răspuns incremental
def call_model_stream(text_widget, header=None):
    try:
        if header:
            message_queue.put(("insert", text_widget, header))

        stream = ollama.chat(
            model='mistral',
            messages=conversation_history,
            stream=True
        )
        assistant_resp = ''
        for chunk in stream:
            if stop_event.is_set():
                break
            part = chunk.get('message', {}).get('content', '')
            assistant_resp += part
            message_queue.put(("insert", text_widget, part))
        # După streaming complet, adăugăm în istoric
        if not stop_event.is_set():
            conversation_history.append({'role': 'assistant', 'content': assistant_resp})
            message_queue.put(("insert", text_widget, "\n"))
            save_conversation()
    except Exception as e:
        message_queue.put(("insert", text_widget, f"Eroare la streaming: {e}\n"))

# Funcție pentru procesarea mesajelor din coadă
def process_queue():
    try:
        while True:
            action, widget, *args = message_queue.get_nowait()
            if action == "insert":
                widget.insert(tk.END, *args)
                widget.see(tk.END)
            elif action == "delete":
                widget.delete(1.0, tk.END)
            message_queue.task_done()
    except queue.Empty:
        pass
    root.after(100, process_queue)

# Configurarea interfeței Tkinter
root = tk.Tk()
root.title("Analiza fișier & Chatbot Malware")

# Frame pentru încărcare fișier
file_frame = ttk.Frame(root)
file_frame.pack(fill='x', pady=5)

load_btn = ttk.Button(file_frame, text="Încarcă fișier", command=load_file)
load_btn.pack(side='left', padx=5)

clear_btn = ttk.Button(file_frame, text="Șterge istoric", command=clear_history)
clear_btn.pack(side='left', padx=5)

stop_btn = ttk.Button(file_frame, text="Oprire conversație", command=stop_conversation)
stop_btn.pack(side='left', padx=5)

# Zonă text pentru analiza fișierului
analysis_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=80, height=10)
analysis_text.pack(padx=10, pady=5)

# Linie de separație
separator = ttk.Separator(root, orient='horizontal')
separator.pack(fill='x', pady=10)

# Frame pentru chatbot
chat_frame = ttk.Frame(root)
chat_frame.pack(fill='both', expand=True)

chat_history = scrolledtext.ScrolledText(chat_frame, wrap=tk.WORD, height=10)
chat_history.pack(fill='both', expand=True, padx=10, pady=5)

entry_frame = ttk.Frame(chat_frame)
entry_frame.pack(fill='x', padx=10, pady=5)

chat_entry = ttk.Entry(entry_frame)
chat_entry.pack(side='left', fill='x', expand=True, padx=(0,5))

send_btn = ttk.Button(entry_frame, text="Trimite", command=send_chat)
send_btn.pack(side='right')

# Start processing the message queue
root.after(100, process_queue)
root.mainloop()
